In [1]:
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from matplotlib.ticker import MaxNLocator

import pandas as pd
import numpy as np
from tqdm.auto import tqdm
from sklearn.preprocessing import MinMaxScaler

import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from multiprocessing import cpu_count
#from torchmetrics.functional import accuracy
from sklearn.metrics import classification_report, confusion_matrix
import math

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
%matplotlib inline
%config InlineBackend.figure_format='retina'

sns.set(style='whitegrid', palette='muted', font_scale=1.2)

HAPPY_COLORS_PALETTE = ["#01BEFE", "#FFDD00", "#FF7D00", "#FF006D", "#93D30C", "#8F00FF"]

sns.set_palette(sns.color_palette(HAPPY_COLORS_PALETTE))

rcParams['figure.figsize'] = 6, 4

tqdm.pandas()


In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [4]:
torch.manual_seed(42)
np.random.seed(42)

In [5]:
df = pd.read_csv("//kaggle/input/upfall-preprocessed/upfall_preprocessed.csv")
df.head()

,ankle_x_axis_g,ankle_y_axis_g,ankle_z_axis_g,ankle_x_axis_deg,ankle_y_axis_deg,ankle_z_axis_deg,belt_x_axis_g,belt_y_axis_g,belt_z_axis_g,belt_x_axis_deg,...,neck_x_axis_deg,neck_y_axis_deg,neck_z_axis_deg,wrist_x_axis_g,wrist_y_axis_g,wrist_z_axis_g,wrist_x_axis_deg,wrist_y_axis_deg,wrist_z_axis_deg,Activity
0,-0.194643,0.079750,-0.013037,-0.144679,-0.015957,0.030258,0.035405,0.092977,0.158983,-0.071006,...,0.060005,0.050541,0.126113,-0.324404,-0.184106,0.232108,-0.193744,-0.122595,0.213697,1
1,-0.194643,0.079554,-0.012869,-0.147716,-0.017521,0.034614,0.035405,0.092977,0.158983,-0.069667,...,0.054218,0.050404,0.121558,-0.324404,-0.184106,0.232108,-0.158895,-0.112007,0.244362,1
2,-0.194643,0.080141,-0.012364,-0.146839,-0.018808,0.033572,0.040892,0.093157,0.159936,-0.070026,...,0.046956,0.044641,0.120674,-0.426488,-0.206134,0.235225,-0.117411,-0.082990,0.274630,1
3,-0.194643,0.080141,-0.012364,-0.146771,-0.020004,0.032530,0.037805,0.092977,0.159777,-0.069765,...,0.046355,0.049279,0.116256,-0.426488,-0.206134,0.235225,-0.135467,-0.054925,0.257740,1
4,-0.195408,0.079750,-0.011187,-0.147851,-0.019820,0.033951,0.037805,0.092977,0.159777,-0.069145,...,0.067322,0.056509,0.112244,-0.380994,-0.199187,0.217171,-0.130185,-0.134862,0.008245,1


In [6]:
df.columns

Index(['ankle_x_axis_g', 'ankle_y_axis_g', 'ankle_z_axis_g',
       'ankle_x_axis_deg', 'ankle_y_axis_deg', 'ankle_z_axis_deg',
       'belt_x_axis_g', 'belt_y_axis_g', 'belt_z_axis_g', 'belt_x_axis_deg',
       'belt_y_axis_deg', 'belt_z_axis_deg', 'neck_x_axis_g', 'neck_y_axis_g',
       'neck_z_axis_g', 'neck_x_axis_deg', 'neck_y_axis_deg',
       'neck_z_axis_deg', 'wrist_x_axis_g', 'wrist_y_axis_g', 'wrist_z_axis_g',
       'wrist_x_axis_deg', 'wrist_y_axis_deg', 'wrist_z_axis_deg', 'Activity'],
      dtype='object')

In [7]:
df.shape

(294678, 25)

In [8]:
target_counts = df['Activity'].value_counts()

In [9]:
positive_percentage = target_counts[1]/df.shape[0]
negative_percentage = target_counts[0]/df.shape[0]
print(f'positive class percentage: {positive_percentage}')
print(f'negative class percentage: {negative_percentage}')

positive class percentage: 0.1559363101419176
negative class percentage: 0.8440636898580823


In [10]:
N_FEATURES = len(df.columns) - 1

In [11]:
def create_sequences(input_data: pd.DataFrame, target_column, sequence_length):

  sequences = []
  data_size = len(input_data)
  STRIDE = 8

  for i in tqdm(range(0, (data_size - sequence_length), STRIDE)):

    sequence = input_data[i:i+sequence_length]

    label_position = i + sequence_length
    label = input_data.iloc[label_position][target_column]
    

    sequence = sequence.drop(['Activity'], axis = 1).T
    sequences.append((sequence, label))

  return sequences

In [12]:
SEQUENCE_LENGTH = 20

sequences = create_sequences(df, 'Activity', SEQUENCE_LENGTH)

  0%|          | 0/36833 [00:00<?, ?it/s]

# Dataset

In [13]:
class ActivityDataset(Dataset):
  def __init__(self, sequences):
    self.sequences = sequences
  def __len__(self):
    return len(self.sequences)
  def __getitem__(self, idx):
    sequence, label = self.sequences[idx]
    sequence=torch.Tensor(sequence.to_numpy())
    label=torch.tensor(label).long()
                                     
    return sequence, label

In [14]:
data = ActivityDataset(sequences)

In [15]:
BATCH_SIZE = 128

# Model

In [16]:
class FCNModel(nn.Module):
    def __init__(self, filters1, filters2, size1, size2):
        super(FCNModel, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=N_FEATURES, out_channels=filters1, kernel_size=size1)
        self.conv2 = nn.Conv1d(in_channels=filters1, out_channels=filters2, kernel_size=size2)
        self.batch_norm1 = nn.BatchNorm1d(filters1)
        self.batch_norm2 = nn.BatchNorm1d(filters2)
        self.relu = nn.ReLU()

        # Calculate output size of conv2 layer
        output_size_conv1 = self.calculate_conv1d_output_size(N_FEATURES, size1)
        output_size_conv2 = self.calculate_conv1d_output_size(output_size_conv1-4, size2)# -4 is bug fix, works for every filter size
            
        self.output_layer = nn.Linear(output_size_conv2, 2)

    def calculate_conv1d_output_size(self, input_size, kernel_size, stride=1, padding=0):
        return ((input_size - kernel_size + 2 * padding) // stride) + 1


    def forward(self, x):
        out = self.relu(self.conv1(x))
        out = self.batch_norm1(out)
        out = self.relu(self.conv2(out))
        out = self.batch_norm2(out)
        out = torch.mean(out, 1)
        out = self.output_layer(out)
        return out

In [17]:
model = FCNModel(128, 128, 3, 3)
model.to(device)

FCNModel(
  (conv1): Conv1d(24, 128, kernel_size=(3,), stride=(1,))
  (conv2): Conv1d(128, 128, kernel_size=(3,), stride=(1,))
  (batch_norm1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batch_norm2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU()
  (output_layer): Linear(in_features=16, out_features=2, bias=True)
)

In [18]:
from torchinfo import summary
summary(model, input_size=(1, 24, 20))

Layer (type:depth-idx)                   Output Shape              Param #
FCNModel                                 [1, 2]                    --
├─Conv1d: 1-1                            [1, 128, 18]              9,344
├─ReLU: 1-2                              [1, 128, 18]              --
├─BatchNorm1d: 1-3                       [1, 128, 18]              256
├─Conv1d: 1-4                            [1, 128, 16]              49,280
├─ReLU: 1-5                              [1, 128, 16]              --
├─BatchNorm1d: 1-6                       [1, 128, 16]              256
├─Linear: 1-7                            [1, 2]                    34
Total params: 59,170
Trainable params: 59,170
Non-trainable params: 0
Total mult-adds (M): 0.96
Input size (MB): 0.00
Forward/backward pass size (MB): 0.07
Params size (MB): 0.24
Estimated Total Size (MB): 0.31

In [19]:
!pip install thop

In [20]:
from thop import profile

input = torch.randn(1, N_FEATURES, 20).to(device)
macs, params = profile(model.to(device), inputs=(input, ))

[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv1d'>.
[INFO] Register count_normalization() for <class 'torch.nn.modules.batchnorm.BatchNorm1d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.activation.ReLU'>.
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.


In [21]:
print(f'macs count: {macs}')
print(f'parameters count: {params}')

macs count: 969760.0
parameters count: 59170.0


In [22]:
def reset_weights(m):
  '''
    Try resetting model weights to avoid
    weight leakage.
  '''
  for layer in m.children():
    if hasattr(layer, 'reset_parameters'):
      layer.reset_parameters()

# Loss function

In [23]:
weight = torch.Tensor([positive_percentage, negative_percentage]).to(device)
loss_fn =  nn.CrossEntropyLoss(weight=weight)

## Early Stopping

In [24]:
class Early_stopper():
    def __init__(self, patience = 1, min_delta = 0):
        self.min_validation = float('inf')
        self.patience = patience
        self.counter = 0
        self.min_delta = min_delta
        
    def early_stop(self, validation):
        if  validation < self.min_validation:
            self.min_validation = validation
            self.counter = 0
            
        elif validation >= (self.min_validation + self.min_delta):
            self.counter += 1
            if self.counter >= self.patience:
                return True
        return False

# Training

In [25]:
from sklearn.model_selection import KFold

In [26]:
def get_confusion_matrix_values(preds, labels):
    
    cm = confusion_matrix(labels, preds, labels=[0, 1])

    TN = cm[0][0]
    TP = cm[1][1]
    FP = cm[0][1]
    FN = cm[1][0]
    
    return TP, TN, FP, FN

In [27]:
def get_accuracy(TP, TN, FP, FN):
    return (TP+TN)/(TP+TN+FP+FN)*100

def get_specificity(TP, TN, FP, FN):
    return TN/(TN+FP)*100

def get_sensitivity(TP, TN, FP, FN):
    return TP/(TP+FN)*100

def get_precision(TP, TN, FP, FN):
    return TP/(TP+FP)*100

def get_f1_score(TP, TN, FP, FN):
    precision = TP/(TP+FP)
    sensitivity = TP/(TP+FN)
    return 2*(precision*sensitivity)/(precision+sensitivity)*100

In [28]:
from torch.optim.lr_scheduler import StepLR

In [29]:
def training_step(training_loader):
    
    t_loss, t_accuracy, t_sensitivity, t_specificity, t_precision, t_f1_score = 0, 0, 0, 0, 0, 0  

    # loop through training batches
    for batch, (X, y) in enumerate(training_loader):

        # activate gradient tracking
        model.train()

        inputs, labels = X.to(device), y.to(device)

        # forward pass
        output = model(inputs)

        preds = torch.softmax(output, dim=1).argmax(dim=1)

        loss = loss_fn(output, labels)
        t_loss += loss

        TP, TN, FP, FN = get_confusion_matrix_values(preds.cpu(), labels.cpu())

        t_accuracy += get_accuracy(TP, TN, FP, FN)
        t_sensitivity += get_sensitivity(TP, TN, FP, FN)
        t_specificity += get_specificity(TP, TN, FP, FN)
        t_precision += get_precision(TP, TN, FP, FN)
        t_f1_score += get_f1_score(TP, TN, FP, FN)

        optimizer.zero_grad()

        loss.backward()

        optimizer.step()
        
    m = len(training_loader)


    return t_loss/m, t_accuracy/m, t_sensitivity/m, t_specificity/m, t_precision/m, t_f1_score/m

In [30]:
def validation_step(validation_loader):
    
    v_loss, v_accuracy, v_sensitivity, v_specificity, v_precision, v_f1_score = 0, 0, 0, 0, 0, 0  
    model.eval()
    with torch.inference_mode():
        for X_val, y_val in validation_loader:

            vinputs, vlabels = X_val.to(device), y_val.to(device)

            voutputs = model(vinputs)
            vpreds = torch.softmax(voutputs, dim=1).argmax(dim=1)

            vloss = loss_fn(voutputs, vlabels)
            v_loss += vloss

            TP, TN, FP, FN = get_confusion_matrix_values(vpreds.cpu(), vlabels.cpu())

            v_accuracy += get_accuracy(TP, TN, FP, FN)
            v_sensitivity += get_sensitivity(TP, TN, FP, FN)
            v_specificity += get_specificity(TP, TN, FP, FN)
            v_precision += get_precision(TP, TN, FP, FN)
            v_f1_score += get_f1_score(TP, TN, FP, FN)
            
    m = len(validation_loader)

    return v_loss/m, v_accuracy/m, v_sensitivity/m, v_specificity/m, v_precision/m, v_f1_score/m


In [31]:
def print_fold_results(results, metric):
    sum = 0.0
    print("")
    print(f"{metric}: ")
    for key, value in results.items():
      print(f'Fold {key}: {value[-1]}')
      sum += value[-1]
    print(f'Average {metric.lower()}: {sum/len(results.items())}')


In [32]:
import time

In [33]:
import warnings
warnings.filterwarnings('ignore')

In [34]:
EPOCHS = 100
K_FOLDS = 5

#for fold results
t_loss_results = {}
t_accuracy_results = {}
t_sensitivity_results = {}
t_specificity_results = {}
t_precision_results = {}
t_f1_score_results = {}
t_time_results = {}

v_loss_results = {}
v_accuracy_results = {}
v_sensitivity_results = {}
v_specificity_results = {}
v_precision_results = {}
v_f1_score_results = {}
v_time_results = {}

# Define the K-fold Cross Validator
kfold = KFold(n_splits=K_FOLDS, shuffle=True, random_state = 42)

# K-fold Cross Validation model evaluation
for fold, (train_ids, val_ids) in enumerate(kfold.split(data)):
    
    t_loss_list, t_accuracy_list, t_precision_list, t_sensitivity_list, t_specificity_list, t_f1_score_list, t_time_list = [],[],[],[],[],[],[]
    v_loss_list, v_accuracy_list, v_precision_list, v_sensitivity_list, v_specificity_list, v_f1_score_list, v_time_list = [],[],[],[],[],[],[]

    print('--------------------------------')
    print(f'FOLD {fold}')
    print('--------------------------------')

    # Sample elements randomly from a given list of ids, no replacement.
    train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
    val_subsampler = torch.utils.data.SubsetRandomSampler(val_ids)
        
    # Define data loaders for training and testing data in this fold
    training_loader = torch.utils.data.DataLoader(
                      data, 
                      batch_size=BATCH_SIZE, sampler=train_subsampler)
    validation_loader = torch.utils.data.DataLoader(
                      data,
                      batch_size=BATCH_SIZE, sampler=val_subsampler)
    
    # Init the neural network
    model = FCNModel(128, 128, 3, 3)
    model.to(device)
    model.apply(reset_weights)
    print("model parameters were reset")
    
    # Initialize optimizer
    optimizer = optim.Adam(model.parameters(), lr=0.0005)
    
    #initialize learning rate decay
    scheduler = StepLR(optimizer, step_size=20)
    
    # Initialize early stopper
    early_stopper = Early_stopper(patience = 10)

    for epoch in tqdm(range(EPOCHS)):
        
        t0 = time.time()
        t_loss, t_accuracy, t_sensitivity, t_specificity, t_precision, t_f1_score = training_step(training_loader)
        t_time = time.time() - t0
            
        # store training results per epoch 
        t_loss_list.append(t_loss)
        t_accuracy_list.append(t_accuracy)
        t_precision_list.append(t_precision)
        t_sensitivity_list.append(t_sensitivity)
        t_specificity_list.append(t_specificity)
        t_f1_score_list.append(t_f1_score)
        t_time_list.append(t_time)
        

        t0 = time.time()
        v_loss, v_accuracy, v_sensitivity, v_specificity, v_precision, v_f1_score = validation_step(validation_loader)
        v_time = time.time() - t0
        
        # store validation results per epoch
        v_loss_list.append(v_loss)
        v_accuracy_list.append(v_accuracy)
        v_precision_list.append(v_precision)
        v_sensitivity_list.append(v_sensitivity)
        v_specificity_list.append(v_specificity)
        v_f1_score_list.append(v_f1_score)
        v_time_list.append(v_time)
        
        
        if epoch % 5 == 0:
            print(f"Epoch: {epoch} | Train loss: {t_loss:.4f}, Train acc: {t_accuracy:.2f}% | Val loss: {v_loss:.4f}, Val acc: {v_accuracy:.2f}%")
          
        # decay the learning rate
        scheduler.step()

        if early_stopper.early_stop(v_loss):
            break    
        
        
    # final results per fold
    t_loss_results[fold] = t_loss_list
    t_accuracy_results[fold] = t_accuracy_list
    t_sensitivity_results[fold] = t_sensitivity_list
    t_specificity_results[fold] = t_specificity_list
    t_precision_results[fold] = t_precision_list
    t_f1_score_results[fold] = t_f1_score_list
    t_time_results[fold] = t_time_list


    v_loss_results[fold] = v_loss_list
    v_accuracy_results[fold] = v_accuracy_list
    v_sensitivity_results[fold] = v_sensitivity_list
    v_specificity_results[fold] = v_specificity_list
    v_precision_results[fold] = v_precision_list
    v_f1_score_results[fold] = v_f1_score_list
    v_time_results[fold] = v_time_list

     
    checkpoint = {'model': FCNModel(128, 128, 3, 3),
                  'state_dict': model.state_dict(),
                  'optimizer' : optimizer.state_dict()}
    
    title = "ABNW_" + str(fold) + ".pth"
    torch.save(checkpoint, title) 


# Print fold results
print('--------------------------------')
print(f'K-FOLD CROSS VALIDATION RESULTS FOR {K_FOLDS} FOLDS')
print('')
print('--------------------------------')
print('             TRAIN              ')
print('--------------------------------')
print_fold_results(t_loss_results, "Loss")
print_fold_results(t_accuracy_results, "Accuracy")
print_fold_results(t_sensitivity_results, "Sensitivity")
print_fold_results(t_specificity_results, "Specificity")
print_fold_results(t_precision_results, "Precision")
print_fold_results(t_f1_score_results, "F1_score")

print('--------------------------------')
print('              VAL               ')
print('--------------------------------')
print_fold_results(v_loss_results, "Loss")
print_fold_results(v_accuracy_results, "Accuracy")
print_fold_results(v_sensitivity_results, "Sensitivity")
print_fold_results(v_specificity_results, "Specificity")
print_fold_results(v_precision_results, "Precision")
print_fold_results(v_f1_score_results, "F1_score")

--------------------------------
FOLD 0
--------------------------------
model parameters were reset


  0%|          | 0/100 [00:00<?, ?it/s]

Epoch: 0 | Train loss: 0.3739, Train acc: 84.79% | Val loss: 0.2271, Val acc: 92.75%
Epoch: 5 | Train loss: 0.0719, Train acc: 98.00% | Val loss: 0.0664, Val acc: 98.29%
Epoch: 10 | Train loss: 0.0447, Train acc: 98.81% | Val loss: 0.0426, Val acc: 99.02%
Epoch: 15 | Train loss: 0.0330, Train acc: 99.16% | Val loss: 0.0290, Val acc: 99.30%
Epoch: 20 | Train loss: 0.0172, Train acc: 99.59% | Val loss: 0.0194, Val acc: 99.62%
Epoch: 25 | Train loss: 0.0122, Train acc: 99.72% | Val loss: 0.0174, Val acc: 99.58%
Epoch: 30 | Train loss: 0.0110, Train acc: 99.76% | Val loss: 0.0174, Val acc: 99.65%
Epoch: 35 | Train loss: 0.0104, Train acc: 99.81% | Val loss: 0.0162, Val acc: 99.68%
Epoch: 40 | Train loss: 0.0085, Train acc: 99.82% | Val loss: 0.0174, Val acc: 99.72%
--------------------------------
FOLD 1
--------------------------------
model parameters were reset


  0%|          | 0/100 [00:00<?, ?it/s]

Epoch: 0 | Train loss: 0.4669, Train acc: 70.82% | Val loss: 0.3042, Val acc: 87.01%
Epoch: 5 | Train loss: 0.0727, Train acc: 97.92% | Val loss: 0.0850, Val acc: 98.62%
Epoch: 10 | Train loss: 0.0577, Train acc: 98.24% | Val loss: 0.0702, Val acc: 98.87%
Epoch: 15 | Train loss: 0.0388, Train acc: 98.78% | Val loss: 0.0473, Val acc: 98.75%
Epoch: 20 | Train loss: 0.0240, Train acc: 99.37% | Val loss: 0.0340, Val acc: 99.39%
Epoch: 25 | Train loss: 0.0171, Train acc: 99.60% | Val loss: 0.0283, Val acc: 99.39%
Epoch: 30 | Train loss: 0.0170, Train acc: 99.60% | Val loss: 0.0284, Val acc: 99.47%
Epoch: 35 | Train loss: 0.0164, Train acc: 99.59% | Val loss: 0.0288, Val acc: 99.61%
--------------------------------
FOLD 2
--------------------------------
model parameters were reset


...

Epoch: 0 | Train loss: 0.4628, Train acc: 68.96% | Val loss: 0.3157, Val acc: 85.98%
Epoch: 5 | Train loss: 0.0786, Train acc: 97.72% | Val loss: 0.0724, Val acc: 98.20%
Epoch: 10 | Train loss: 0.0553, Train acc: 98.39% | Val loss: 0.0785, Val acc: 96.44%
Epoch: 15 | Train loss: 0.0375, Train acc: 98.96% | Val loss: 0.0396, Val acc: 99.15%
Epoch: 20 | Train loss: 0.0231, Train acc: 99.37% | Val loss: 0.0325, Val acc: 99.22%
Epoch: 25 | Train loss: 0.0172, Train acc: 99.58% | Val loss: 0.0375, Val acc: 99.36%
Epoch: 30 | Train loss: 0.0163, Train acc: 99.63% | Val loss: 0.0314, Val acc: 99.41%
Epoch: 35 | Train loss: 0.0140, Train acc: 99.65% | Val loss: 0.0340, Val acc: 99.41%
--------------------------------
FOLD 3
--------------------------------
model parameters were reset


  0%|          | 0/100 [00:00<?, ?it/s]

Epoch: 0 | Train loss: 0.4418, Train acc: 73.79% | Val loss: 0.2975, Val acc: 86.52%
Epoch: 5 | Train loss: 0.0721, Train acc: 97.93% | Val loss: 0.1281, Val acc: 97.78%
Epoch: 10 | Train loss: 0.0471, Train acc: 98.66% | Val loss: 0.0868, Val acc: 96.36%
Epoch: 15 | Train loss: 0.0398, Train acc: 98.80% | Val loss: 0.0629, Val acc: 98.66%
Epoch: 20 | Train loss: 0.0195, Train acc: 99.51% | Val loss: 0.0356, Val acc: 99.16%
Epoch: 25 | Train loss: 0.0165, Train acc: 99.61% | Val loss: 0.0355, Val acc: 99.34%
Epoch: 30 | Train loss: 0.0147, Train acc: 99.66% | Val loss: 0.0379, Val acc: 99.34%
Epoch: 35 | Train loss: 0.0130, Train acc: 99.68% | Val loss: 0.0316, Val acc: 99.37%
Epoch: 40 | Train loss: 0.0123, Train acc: 99.71% | Val loss: 0.0317, Val acc: 99.27%
Epoch: 45 | Train loss: 0.0123, Train acc: 99.70% | Val loss: 0.0337, Val acc: 99.34%
Epoch: 50 | Train loss: 0.0108, Train acc: 99.76% | Val loss: 0.0328, Val acc: 99.41%
--------------------------------
FOLD 4
----------------

  0%|          | 0/100 [00:00<?, ?it/s]

Epoch: 0 | Train loss: 0.4340, Train acc: 71.48% | Val loss: 0.2219, Val acc: 95.66%
Epoch: 5 | Train loss: 0.0628, Train acc: 98.42% | Val loss: 0.1053, Val acc: 95.44%
Epoch: 10 | Train loss: 0.0444, Train acc: 98.83% | Val loss: 0.0480, Val acc: 98.81%
Epoch: 15 | Train loss: 0.0295, Train acc: 99.26% | Val loss: 0.0445, Val acc: 99.06%
Epoch: 20 | Train loss: 0.0151, Train acc: 99.67% | Val loss: 0.0265, Val acc: 99.46%
Epoch: 25 | Train loss: 0.0106, Train acc: 99.77% | Val loss: 0.0251, Val acc: 99.52%
Epoch: 30 | Train loss: 0.0099, Train acc: 99.75% | Val loss: 0.0239, Val acc: 99.54%
Epoch: 35 | Train loss: 0.0086, Train acc: 99.81% | Val loss: 0.0277, Val acc: 99.50%
--------------------------------
K-FOLD CROSS VALIDATION RESULTS FOR 5 FOLDS

--------------------------------
             TRAIN              
--------------------------------

Loss: 
Fold 0: 0.008068440482020378
Fold 1: 0.01461064349859953
Fold 2: 0.013830084353685379
Fold 3: 0.010782838799059391
Fold 4: 0.0079